In [ ]:
pip install tweepy
pip install langdetect

In [ ]:
import pandas as pd
import tweepy as tw
import string
import nltk
from langdetect import detect
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
nltk.download('wordnet')

In [ ]:
###### DATA MINING ######

#variable names you need to set to your own
datafile = "2020-03-31-dataset.tsv"
name_of_file = "contentOfTweet_mar31"

#These are necassery to access the twitterAPI
API_KEY = "UgIsUxFWl6jPmGALU5oISJ4CD"
API_SECRET = "tEpI8kTz3sITys0dS4VESn3VuMi6ca6D9aBjCfYJws6f6QxHa3"
ACCESS_TOKEN = "1260975277343195139-kmZlvSlyoRUbkwkmy6dsX6SBteXO4Z"
ACCESS_TOKEN_SECRET = "6VRmGTFNxbAqogTKXirDEi9y3KIUwiFtnhhAXK2kzAPsQ"

#Load in the specific dataset
all_tweets = pd.read_csv(datafile, sep='\t')

#Upper bound on the amount of tweets
max_tweets = all_tweets.shape[0]

#auth = tw.AppAuthHandler(API_KEY, API_SECRET)
auth = tw.OAuthHandler(API_KEY, API_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tw.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

#check if there are more than 50000 tweets in the dataset
#if there is randomize the order of the tweets
if all_tweets.shape[0] > 30000:
    all_tweets = all_tweets.sample(frac=1).reset_index(drop=True)
    max_tweets = 30000
    
#get the text of the tweet per id  
for index in range(max_tweets):
    tid = all_tweets.iloc[index]['tweet_id']
    try:
        tweet    = api.get_status(tid) 
        tweetxt  = tweet.text
        language = detect(tweetxt)
        all_tweets.loc[all_tweets.index[index], 'content']  = tweetxt
        all_tweets.loc[all_tweets.index[index], 'language'] = language
    
    #For if the tweet has been deleted
    except: 
        all_tweets.loc[all_tweets.index[index], 'content'] = 'deleted'
      
    if (index % 100) == 0:
        print("{} {}".format(index, "tweets done"))
        
#save to a csv file        
all_tweets.to_csv(name_of_file)


###### SENTIMENT ANALYSIS ######

#name of dedicated files
file = "contentOfTweet_apr21"
sentimentfile = 'apr21_with_sentiment'

#open file with tweets
all_languages = pd.read_csv(file, sep=',')

#Open the sentiment lexicon as a dataframe
lexicon = pd.read_csv ('sentiment_lexicon.csv', sep=';')

#Make a set of words from the lexicon
word_list = set(lexicon['word'])

#filter the tweets on english only
only_english = all_languages.loc[all_languages['language'] == 'en']
only_english = only_english.reset_index(drop=True)

print('Done with filtering on english')

lemmatizer = WordNetLemmatizer()
cachedStopWords = stopwords.words("english")

for indx, row in only_english.iterrows():
    twt = only_english.iloc[indx]['content']
    twt = ' '.join(word for word in twt.split(' ') if not word.startswith('http')) #Remove URL's
    twt = ' '.join(word for word in twt.split(' ') if not word.startswith('@'))    #Remove @ with the user ID
    twt = twt.replace('RT', '')                                                    #Remove the RT statement
    
    #POS tag the sentence
    tex  = word_tokenize(twt)
    tags = nltk.pos_tag(tex)
    newlist = []
    
    for word, tag in tags:
        
        #If the word is a stop word do nothing
        if not word in cachedStopWords:
            
            #If the word is not in the lexicon we check if the word's stem IS in the lexicon, if so we replace the word with it's stem
            if not word in word_list:

                #Take the stem if the word is verb
                if tag in ('VBD','VBG','VBN','VBP','VBZ'):
                    vstem = lemmatizer.lemmatize(word,'v')
                    newlist.append(vstem)
                
                #Take the stem if the word is a nn
                elif tag in ('NNS', 'NNPS'):
                    nstem = lemmatizer.lemmatize(word, 'n')
                    newlist.append(nstem)
                    
            #if the word is in the lexicon then we don't need to alter the word form
            else:
                    newlist.append(word)
    
    #Make a string from the list of words
    twt = " ".join(newlist)
    
    twt = twt.translate(str.maketrans('', '', string.punctuation))        #Remove puntuations
    only_english.loc[only_english.index[indx], 'content']  = twt.lower()  #lower case everything

print('Done with NLP')
    
#Add new columns for every emotions to be detected
number_of_rowsZero = [0]*only_english.shape[0]
only_english['anger']        = number_of_rowsZero
only_english['anticipation'] = number_of_rowsZero
only_english['disgust']      = number_of_rowsZero
only_english['fear']         = number_of_rowsZero
only_english['joy']          = number_of_rowsZero
only_english['sadness']      = number_of_rowsZero
only_english['surprise']     = number_of_rowsZero
only_english['trust']        = number_of_rowsZero

#Loop over the tweet dataframe
#Get the content of the tweet and put it into a list of words
for index, row in only_english.iterrows():
    twt = only_english.iloc[index]['content']
    twt = twt.split()
    
    #Check if the word of the tweet can be found somewhere in the sentiment lexicon
    for word in twt:
        #When the word is not in the list it does not have any sentiment and thus nothing to do with it
        if word in word_list:
            word_index = lexicon[lexicon['word']==word].index.values.astype(int)[0]
            emotions   = lexicon.iloc[word_index]['emotion']
            score      = lexicon.iloc[word_index]['emotion-intensity-score']

            #Check to which emotion the word corresponds and add up the score of that word to the already 
            #excisting score of the tweet
            if emotions   == 'anger':
                score_so_far = only_english.iloc[index]['anger']
                only_english.loc[only_english.index[index], 'anger']  = score_so_far + score
            elif emotions == 'anticipation':
                score_so_far = only_english.iloc[index]['anticipation']
                only_english.loc[only_english.index[index], 'anticipation']  = score_so_far + score
            elif emotions == 'disgust':
                score_so_far = only_english.iloc[index]['disgust']
                only_english.loc[only_english.index[index], 'disgust']  = score_so_far + score
            elif emotions == 'fear':
                score_so_far = only_english.iloc[index]['fear']
                only_english.loc[only_english.index[index], 'fear']  = score_so_far + score
            elif emotions == 'joy':
                score_so_far = only_english.iloc[index]['joy']
                only_english.loc[only_english.index[index], 'joy']  = score_so_far + score
            elif emotions == 'sadness':
                score_so_far = only_english.iloc[index]['sadness']
                only_english.loc[only_english.index[index], 'sadness']  = score_so_far + score
            elif emotions == 'surprise':
                score_so_far = only_english.iloc[index]['surprise']
                only_english.loc[only_english.index[index], 'surprise']  = score_so_far + score
            else:
                score_so_far = only_english.iloc[index]['trust']
                only_english.loc[only_english.index[index], 'trust']  = score_so_far + score

print('Done with emotion detection')
                
#Save the dataframe with sentiment score in a new file
only_english.to_csv(sentimentfile)

###### DATA PREPROCESSING FOR VISUALIZATION ######

sentiment_aprl = pd.read_csv('Sentiment_FebMarchAprilMay', sep=',')

date_and_sentiment = sentiment_aprl[['date','anger', 'anticipation', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'trust']]

av_per_day = date_and_sentiment.groupby(['date'], as_index=False).mean()


for index, row in av_per_day.iterrows():
    datestring = row['date']
    month      = datestring[5]
    day        = int(datestring[-2:])
    
    if day < 0:
        day = day *-1
        
    av_per_day.loc[index, 'day'] = day
    av_per_day.loc[index, 'month']  = month
av_per_day.sort_values(by=['month', 'day'], inplace=True)  
av_per_day = av_per_day.reset_index(drop=True)
print(av_per_day)

###### VISUALIZATION ######

av_per_day.plot.line(x= 'date', y=['anger', 'anticipation', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'trust'])
plt.title('Mean sentiments per day')
plt.ylabel('Sentiment score')
plt.xlabel('Date')
plt.xticks(rotation= 65)
plt.legend(loc=(1.05, 0.4))
